In [68]:
import string
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [69]:
df = pd.read_csv("books_dataframe_cleaned.csv")
df.head()


,name,price,nb_in_stock,url_img,product_category,rating,description,unique_id
0,It's Only the Himalayas,45.17,19,https://books.toscrape.com/catalogue/category/...,travel_2,2.0,"<p>âWherever you go, whatever you do, just ....",21ac9fe7
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,15,https://books.toscrape.com/catalogue/category/...,travel_2,4.0,<p>Acclaimed travel writer Rick Antonson sets ...,01ffad8c
2,See America: A Celebration of Our National Par...,48.87,14,https://books.toscrape.com/catalogue/category/...,travel_2,3.0,<p>To coincide with the 2016 centennial annive...,833d32d1
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,8,https://books.toscrape.com/catalogue/category/...,travel_2,2.0,<p>With a new foreword by Tim Ferriss â¢There...,d091fe77
4,Under the Tuscan Sun,37.33,7,https://books.toscrape.com/catalogue/category/...,travel_2,3.0,<p>A CLASSIC FROM THE BESTSELLING AUTHOR OF UN...,0f6866a5


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              1000 non-null   object 
 1   price             1000 non-null   float64
 2   nb_in_stock       1000 non-null   int64  
 3   url_img           1000 non-null   object 
 4   product_category  1000 non-null   object 
 5   rating            1000 non-null   float64
 6   description       1000 non-null   object 
 7   unique_id         1000 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 62.6+ KB


In [74]:
def nettoyer_df(df):
    if df.isnull().values.any():
        df = df.dropna()
    return df
df=nettoyer_df(df)

In [75]:
####Tokenisation
def nettoyer_et_tokenizer(texte):
    # 1. Mise en minuscules
    texte = texte.lower()
    
    # 2. Suppression de la ponctuation
    # On crée une table de traduction qui remplace la ponctuation par rien
    ponctuation= str.maketrans('', '', string.punctuation)
    texte_sans_ponct = texte.translate(ponctuation)
    
    # 3. Tokenisation par découpage sur les espaces
    tokens = texte_sans_ponct.split()
    
    return tokens
df['tokens']=df['description'].apply(nettoyer_et_tokenizer)

In [76]:
#df.keys()

Index(['name', 'price', 'nb_in_stock', 'url_img', 'product_category', 'rating',
       'description', 'unique_id', 'tokens'],
      dtype='object')

In [77]:
#df.description.loc[5]

'<p>On her thirtieth birthday, Gwendolyn Reese receives an unexpected present from her widowed Aunt Bea: a grand tour of Europe in the company of Bea\'s Sudoku and Mahjongg Club. The prospect isn\'t entirely appealing. But when the gift she is expecting--an engagement ring from her boyfriend--doesn\'t materialize, Gwen decides to go. At first, Gwen approaches the trip as if it\'s On her thirtieth birthday, Gwendolyn Reese receives an unexpected present from her widowed Aunt Bea: a grand tour of Europe in the company of Bea\'s Sudoku and Mahjongg Club. The prospect isn\'t entirely appealing. But when the gift she is expecting--an engagement ring from her boyfriend--doesn\'t materialize, Gwen decides to go. At first, Gwen approaches the trip as if it\'s the math homework she assigns her students, diligently checking monuments off her must-see list. But amid the bougainvillea and stunning vistas of southern Italy, something changes. Gwen begins to live in the moment--skipping down stone s

In [78]:
#nettoyer_et_tokenizer(df.description.loc[5])

['pon',
 'her',
 'thirtieth',
 'birthday',
 'gwendolyn',
 'reese',
 'receives',
 'an',
 'unexpected',
 'present',
 'from',
 'her',
 'widowed',
 'aunt',
 'bea',
 'a',
 'grand',
 'tour',
 'of',
 'europe',
 'in',
 'the',
 'company',
 'of',
 'beas',
 'sudoku',
 'and',
 'mahjongg',
 'club',
 'the',
 'prospect',
 'isnt',
 'entirely',
 'appealing',
 'but',
 'when',
 'the',
 'gift',
 'she',
 'is',
 'expectingan',
 'engagement',
 'ring',
 'from',
 'her',
 'boyfrienddoesnt',
 'materialize',
 'gwen',
 'decides',
 'to',
 'go',
 'at',
 'first',
 'gwen',
 'approaches',
 'the',
 'trip',
 'as',
 'if',
 'its',
 'on',
 'her',
 'thirtieth',
 'birthday',
 'gwendolyn',
 'reese',
 'receives',
 'an',
 'unexpected',
 'present',
 'from',
 'her',
 'widowed',
 'aunt',
 'bea',
 'a',
 'grand',
 'tour',
 'of',
 'europe',
 'in',
 'the',
 'company',
 'of',
 'beas',
 'sudoku',
 'and',
 'mahjongg',
 'club',
 'the',
 'prospect',
 'isnt',
 'entirely',
 'appealing',
 'but',
 'when',
 'the',
 'gift',
 'she',
 'is',
 'expec

In [79]:
# Créer le vectorizer en supprimant les stop words français
vectorizer = TfidfVectorizer(stop_words='english')

# Appliquer directement sur la colonne 'description' (texte brut)
X = vectorizer.fit_transform(df['description'].astype(str))  # astype(str) pour éviter erreurs si NaN

# Pour voir les mots retenus (sans stop words)
print(vectorizer.get_feature_names_out())


['00' '000' '001' ... 'zy' 'zz' '½collecting']


In [80]:
####Embeding
# 1. Fusionner 'name' et 'description' pour avoir plus de contexte
df["text_final"] = df["description"].astype(str)

# 2. Vectorisation avec TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["text_final"])

# 3. Vérification
#print("Forme de la matrice TF-IDF :", X.shape)
print("Exemple de mots :", vectorizer.get_feature_names_out()[:20])


Exemple de mots : ['00' '000' '001' '00am' '01' '02' '03' '05' '0679720200' '08' '10' '100'
 '100th' '101' '108' '109' '1099' '10â' '11' '110']


In [81]:
vectorizer

TfidfVectorizer(stop_words='english')

In [82]:
vectorizer.get_feature_names_out()

array(['00', '000', '001', ..., 'zy', 'zz', '½collecting'], dtype=object)

In [83]:
pd.DataFrame(vectorizer.inverse_transform(X))

,0,1,2,3,4,5,6,7,8,9,...,441,442,443,444,445,446,447,448,449,450
0,just,donâ,stupid,motherduring,yearlong,adventure,backpacking,south,africa,singapore,...,None,None,None,None,None,None,None,None,None,None
1,great,mountain,joined,world,acclaimed,travel,writer,rick,antonson,sets,...,None,None,None,None,None,None,None,None,None,None
2,world,travel,coincide,2016,centennial,anniversary,national,parks,service,creative,...,None,None,None,None,None,None,None,None,None,None
3,just,life,experience,trip,lifetime,world,discover,travel,writer,rick,...,None,None,None,None,None,None,None,None,None,None
4,did,life,world,travel,author,country,new,book,classic,bestselling,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,did,white,life,way,politics,author,country,historical,engaging,explains,...,None,None,None,None,None,None,None,None,None,None
996,just,discover,history,place,told,known,america,nation,achieve,dream,...,None,None,None,None,None,None,None,None,None,None
997,africa,friends,love,home,young,country,national,service,nation,dream,...,None,None,None,None,None,None,None,None,None,None
998,just,things,moments,want,stolen,like,gift,dark,year,left,...,None,None,None,None,None,None,None,None,None,None
